In [2]:
import glob
import json
import os

import anthropic

In [3]:
conversations = []

paths = glob.glob('./res/297.SNS 데이터 고도화/01-1.정식개방데이터/Validation/02.라벨링데이터/VL/*json')
target_count = 20
count = 0

for path in paths:
    with open(path, 'r') as f:
        conv_dict = json.load(f)
        if conv_dict['header']['dialogueInfo']['numberOfParticipants'] == 2:
            if conv_dict['header']['dialogueInfo']['numberOfUtterances'] > 30:
                conv_list = []
                for d in conv_dict['body']:
                    conv_list.append(d['participantID'] + ': ' + d['utterance'])
                if conv_list[0] == conv_list[1]:
                    print('Repeated Conversations')
                    continue
                conv_text = '\n'.join(conv_list)
                conversations.append(conv_text)
                count += 1
                if count == target_count:
                    break

target_count = 30
count = 0

for path in paths:
    with open(path, 'r') as f:
        conv_dict = json.load(f)
        if conv_dict['header']['dialogueInfo']['numberOfParticipants'] > 2:
            if conv_dict['header']['dialogueInfo']['numberOfUtterances'] > 30:
                conv_list = []
                for d in conv_dict['body']:
                    conv_list.append(d['participantID'] + ': ' + d['utterance'])
                if conv_list[0] == conv_list[1]:           
                    continue
                conv_text = '\n'.join(conv_list)
                conversations.append(conv_text)
                count += 1
                if count == target_count:
                    break

count

30

In [4]:
prompt = f"""You are given a conversation among three users below. Imagine what the users discussed prior to the conversation.
Please write the previous content, strictly following the tone and format of given conversation, with at least 3000 characters.

{conversations[-2]}
"""

In [5]:
print(conversations[-2])

P01: 웹툰이나 만화 자주 보는 편이야?
P02: 나 웹툰은 안 보고 만화는 코난이랑 짱구 슬램덩크 이런 거 좋아해~ 넌?
P03: 난 반대로 만화 안보고 웹툰만 엄청 봐 키키
P03: 요즘에는 시험 때문에 못 보기는 하는데 끝나면 다 몰아볼 예정이야 후욱후욱
P01: 키키 나는 둘 다 안 봄~
P01: 중학교 때는 가끔 만화책 빌려서 봐보긴 했는데 그것도 몇 개 안돼...
P01: 웹툰 재밌어?
P02: 어떤 작가 좋아해?
P02: 특별히 좋아하는 웹툰 있어?
P03: 이동건이라고 유미의 세포들 그린 작가인데 진짜 캐릭처도 귀엽고 감정선이 정말 여자인 건가 싶을 정도로 세심해요!
P03: 재미도 있고 지금도 뭐 신작 나왔더라고!
P01: 아 유미의 세포들! 들어보긴 했어~
P01: 유명하더라 영화인지 드라마로도 나오는 거 같던데? 김고은이 주인공으로
P02: 맞아 드라마 야나? 예고편 같은 거 떴던데~
P03: 웅웅 맞아 맞아 그거 웹툰이 엄청 흥행 해서 드라마화 한다고 하더라...
P03: 근데 개인적으로 웹툰 기반 드라마화 한 거 그렇게 재미는 없을 것 같아서 딱히 기대는 안 하는 중
P01: 맞아 전에 치즈인더트랩도... 별로 였다고 했던 거 같애
P02: 그건 드라마 영화 둘 다 이상해 ㅠㅠ 내용은 괜찮은데~
P03: 맞아 드라마로 했다가 평 안 좋았어서 영화로도 나왔는데 더 망했던 걸로 기억해 키키
P03: 그나마 요번에 한 간 떨어지는 동거 그거는 잘 된 것 같더라고?
P01: 아아 장기용이랑 혜리 나온 거? 뭐 구미호 얘기인 거야...?
P02: 아 진짜? 그것도 앞에만 보다 말았는데~
P03: 웅웅 맞아 맞아 장기용 여기도 나왔네 맞다 키키
P03: 혹시 마블 영화 좋아해?
P01: 보면 재밌는데 나는 막 챙겨보는 편은 아니야! 좋아해?
P02: 안 좋아해~ 다들 재밌다고 해서 맘 잡고 보려고 하는데 눈에 잘 안 들어오더라~ ㅠㅠ
P03: 약간 나랑 언니랑 취향 좀 다른 듯 키키
P03: 나는 마블 영화 엄청 챙겨 보는데 ㅠ
P01: 보면 재

In [8]:
client = anthropic.Anthropic(
    api_key=os.environ['ANTHROPIC_API_KEY']
)

message = client.messages.create(
    model="claude-3-5-sonnet-20240620",
    max_tokens=4096,
    temperature=0.0,
    messages=[
        {"role": "user", "content": prompt}
    ]
)

message.usage  # API 호출 비용 대략 50원

Usage(input_tokens=1415, output_tokens=2938, cache_creation_input_tokens=0, cache_read_input_tokens=0)

In [9]:
message.content[0].text.split('\n\n')

["Here's an imagined previous conversation among the three users, following the tone and format of the given conversation:",
 "P02: 야 너네 요즘 뭐하고 지내?\nP01: 나? 그냥 평소처럼 학교 다니고 공부하고 그러지 뭐\nP03: 나도 비슷해~ 요즘 시험기간이라 공부하느라 정신없어\nP02: 아 맞다 너네 시험기간이었지? 힘내!\nP01: 고마워~ 너는 뭐하고 지내?\nP02: 나는 요즘 드라마 보는 재미에 빠졌어\nP03: 오~ 무슨 드라마 보고 있어?\nP02: 나는 요즘 '펜트하우스' 보고 있어. 진짜 재밌더라고!\nP01: 아 그거 엄청 인기 많았던 거 아니야?\nP03: 맞아맞아! 나도 들어본 것 같아\nP02: 응응, 시즌 3까지 나왔어. 막장이긴 한데 보다 보면 계속 보게 돼\nP01: 키키 그런 거 좋아하네~\nP03: 나는 그런 거 별로 안 좋아해서... 시간 아까워\nP02: 에이~ 한 번 보면 빠질걸?\nP03: 글쎄~ 난 그냥 웹툰이나 보는 게 더 재밌어\nP01: 아 그러고 보니 너 웹툰 좋아하지?\nP03: 응응, 나 웹툰 엄청 좋아해!\nP02: 난 웹툰은 잘 안 봐서... 어떤 게 재밌어?\nP03: 와, 진짜 많은데! '유미의 세포들'이랑 '여신강림' 같은 로맨스부터 '신의 탑' 같은 판타지까지 다양하게 봐\nP01: 와~ 진짜 많이 보는구나\nP02: 그렇게 많이 보면 시간은 언제 나?\nP03: 그냥 틈틈이 보는 거지~ 지하철 타고 갈 때도 보고\nP01: 아~ 그렇구나. 나는 지하철에서 그냥 음악 듣는데\nP02: 나도! 음악 들으면서 풍경 구경하는 게 좋더라고\nP03: 키키 너네 둘이 비슷하네~\nP01: 그러게~ 취향이 비슷한가?\nP02: 음악도 취향이 비슷할라나? 어떤 음악 좋아해?\nP01: 나는 발라드랑 R&B 좋아해\nP03: 오~ 나는 힙합이랑 아이돌 음악!\nP02: 난 클래식이랑 재즈 좋아해~\nP0

In [10]:
lst = message.content[0].text.split('\n\n')[1:-1]
for l in lst[:25]:
    print(l)

In [11]:
for l in lst[25:50]:
    print(l)

In [12]:
print(len(message.content[0].text.split('\n\n')[1]))


2781


In [13]:
lst = message.content[0].text.split('\n\n')[1:-1]
lst

[]

In [14]:
import pickle

with open('./res/conv_long.pickle', 'wb') as f:
    pickle.dump(lst, f)